In [2]:
import numpy as np
import cv2 as cv
import glob
import pathlib

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((5 * 4, 3), np.float32)
objp[:, :2] = np.mgrid[0:5, 0:4].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

In [3]:
images = glob.glob("raw/*")

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (5, 4), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (5, 4), corners2, ret)

        # cv.imshow("img", img)
        path = pathlib.Path(fname)
        dest = f"dots/{path.name}"
        cv.imwrite(dest, img)

        # cv.waitKey(500)

# cv.destroyAllWindows()

KeyboardInterrupt: 

In [5]:
len(objpoints)

30

In [6]:
# chess: 17cm x 14.2cm
# target: 15cm x 15cm

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

In [10]:
display(mtx)
display(dist)
# fx = 3198, fy = 3184
# cx = 1983, cy = 1618

array([[3.19845175e+03, 0.00000000e+00, 1.98341924e+03],
       [0.00000000e+00, 3.18371808e+03, 1.61826118e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

array([[ 0.14704232, -0.5279449 ,  0.00516793, -0.00580421,  0.74135366]])

In [8]:
for fname in glob.glob("raw/*"):
    img = cv.imread(fname)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
    
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]

    path = pathlib.Path(fname)
    dest = f"straight/{path.name}"
    cv.imwrite(dest, dst)
